In [5]:
!ps ax | grep python


  607 ?        Sl     0:04 /usr/bin/python2 /usr/local/bin/jupyter-notebook -y --no-browser --log-level=DEBUG --debug --NotebookApp.allow_origin="*" --NotebookApp.log_format="%(message)s" --NotebookApp.disable_check_xsrf=True --NotebookApp.token= --Session.key="" --Session.keyfile="" --ContentsManager.untitled_directory="Untitled Folder" --ContentsManager.untitled_file="Untitled File" --ContentsManager.untitled_notebook="Untitled Notebook" --KernelManager.autorestart=True --MultiKernelManager.default_kernel_name="python2" --ip="127.0.0.1" --port=9000 --port-retries=0 --notebook-dir="/content" --NotebookApp.base_url=/tun/m/gpu-1fdry24m8530s/
  991 ?        Ssl   53:00 /usr/bin/python3 -m ipykernel_launcher -f /content/.local/share/jupyter/runtime/kernel-c29fd895-bbd0-4856-9914-02e89fa9aa33.json
 1066 pts/0    Ss+    0:01 /bin/sh -c ps ax | grep python
 1068 pts/0    S+     0:00 grep python


In [0]:
!kill -9 991

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.8 GB  | Proc size: 139.0 MB
GPU RAM Free: 11438MB | Used: 1MB | Util   0% | Total 11439MB


In [4]:
from google.colab import files
files.upload()

Saving neural_network_collab.py to neural_network_collab.py


{'neural_network_collab.py': b"import tensorflow as tf\nimport numpy as np\n\nMAX_EXP = 18\n\nclass NeuralNetwork:\n    def __init__(\n            self,\n            activation_function=tf.nn.relu,\n            learning_rate=0.0005,\n            padding='same'\n        ):\n        self.input = tf.placeholder(tf.float32, shape=(None, 4, 4, MAX_EXP), name='input')\n\n        conv_1 = tf.layers.conv2d(\n            inputs=self.input,\n            filters=1000,\n            kernel_size=[4, 1],\n            padding=padding,\n            activation=activation_function\n        )\n\n        conv_2 = tf.layers.conv2d(\n            inputs=conv_1,\n            filters=1000,\n            kernel_size=[1, 4],\n            padding=padding,\n            activation=activation_function\n        )\n\n        conv_3 = tf.layers.conv2d(\n            inputs=conv_2,\n            filters=1000,\n            kernel_size=[4, 1],\n            padding=padding,\n            activation=activation_function\n        

In [3]:
rm neural_network_collab.py

In [2]:
from experience_replay import ExperienceBuffer
import neural_network_collab as dnn
import numpy as np
import game
import random

# Buffer for experience replay
exp_buffer = ExperienceBuffer()

# Hyperparameters
discount_factor = 0.9
learning_rate = 0.0005
epsilon = 1
turns_until_min_epsilon = 1000

# Network
net = dnn.NeuralNetwork(learning_rate=learning_rate)
net.start_session()

# Information about training
number_of_games = 500
batch_size = 3
turn_count = 0

for game_count in range(number_of_games):
    print('Game', game_count + 1)
    state = game.initial_state()
    moved = True
    score = 0
    game_len = 0
    while not game.is_game_over(state):
        action = None
        if moved:
            game_len += 1
            turn_count = min(turn_count + 1, turns_until_min_epsilon)
            epsilon = 1 - 0.95 * turn_count / turns_until_min_epsilon
            state = game.do_a_random_spawn(state)
            if random.random() < epsilon:
                action = random.randrange(game.number_of_actions)
            else:
                action = net.predict([dnn.prepare_input(state)])[1][0]
        else:
            action = random.randrange(game.number_of_actions)

        new_state, moved, reward = game.action(state, action, copy=True)
        score += reward
        if moved:
            exp_buffer.add([dnn.prepare_input(state), action, reward, dnn.prepare_input(new_state)])
            state = new_state
        if not exp_buffer.is_empty():
            batch = exp_buffer.get_batch(batch_size)
            old_states = [i[0] for i in batch]
            new_states = [i[3] for i in batch]
            rewards = [i[2] for i in batch]
            actions = [i[1] for i in batch]
            Q_values, initial_predictions = net.predict(old_states)
            target_Q_values = net.max_Q(new_states) * discount_factor + rewards
            for i in range(batch_size):
                Q_values[i][actions[i]] = target_Q_values[i]
            loss, _ = net.train(old_states, Q_values)
    print('Max tile after game', game_count + 1, 'is', state.max(), 'score is', score, 'and game length is', game_len)


Game 1
Max tile after game 1 is 64.0 score is 840.0 and game length is 103
Game 2
Max tile after game 2 is 32.0 score is 488.0 and game length is 81
Game 3
Max tile after game 3 is 128.0 score is 1568.0 and game length is 166
Game 4
Max tile after game 4 is 64.0 score is 856.0 and game length is 103
Game 5
Max tile after game 5 is 64.0 score is 696.0 and game length is 94
Game 6
Max tile after game 6 is 128.0 score is 1276.0 and game length is 129
Game 7
Max tile after game 7 is 128.0 score is 1036.0 and game length is 112
Game 8
Max tile after game 8 is 128.0 score is 1348.0 and game length is 137
Game 9
Max tile after game 9 is 256.0 score is 2316.0 and game length is 195
Game 10
Max tile after game 10 is 128.0 score is 1024.0 and game length is 113
Game 11
Max tile after game 11 is 128.0 score is 1172.0 and game length is 128
Game 12
Max tile after game 12 is 64.0 score is 660.0 and game length is 93
Game 13
Max tile after game 13 is 64.0 score is 748.0 and game length is 101
Game 1

Max tile after game 51 is 128.0 score is 1492.0 and game length is 154
Game 52
Max tile after game 52 is 64.0 score is 916.0 and game length is 113
Game 53
Max tile after game 53 is 128.0 score is 1392.0 and game length is 144
Game 54
Max tile after game 54 is 128.0 score is 1064.0 and game length is 114
Game 55
Max tile after game 55 is 64.0 score is 756.0 and game length is 99
Game 56
Max tile after game 56 is 128.0 score is 1488.0 and game length is 159
Game 57
Max tile after game 57 is 256.0 score is 2964.0 and game length is 245
Game 58
Max tile after game 58 is 32.0 score is 408.0 and game length is 70
Game 59
Max tile after game 59 is 64.0 score is 604.0 and game length is 84
Game 60
Max tile after game 60 is 128.0 score is 996.0 and game length is 107
Game 61
Max tile after game 61 is 64.0 score is 500.0 and game length is 73
Game 62
Max tile after game 62 is 64.0 score is 768.0 and game length is 104
Game 63
Max tile after game 63 is 128.0 score is 1408.0 and game length is 14

Max tile after game 101 is 64.0 score is 528.0 and game length is 78
Game 102
Max tile after game 102 is 32.0 score is 392.0 and game length is 68
Game 103
Max tile after game 103 is 128.0 score is 1212.0 and game length is 131
Game 104
Max tile after game 104 is 64.0 score is 892.0 and game length is 117
Game 105
Max tile after game 105 is 128.0 score is 1720.0 and game length is 176
Game 106
Max tile after game 106 is 64.0 score is 620.0 and game length is 89
Game 107
Max tile after game 107 is 128.0 score is 1328.0 and game length is 139
Game 108
Max tile after game 108 is 64.0 score is 804.0 and game length is 102
Game 109
Max tile after game 109 is 128.0 score is 1464.0 and game length is 157
Game 110
Max tile after game 110 is 256.0 score is 2816.0 and game length is 234
Game 111
Max tile after game 111 is 128.0 score is 1416.0 and game length is 147
Game 112
Max tile after game 112 is 128.0 score is 1632.0 and game length is 161
Game 113
Max tile after game 113 is 64.0 score is 

Max tile after game 150 is 64.0 score is 588.0 and game length is 81
Game 151
Max tile after game 151 is 64.0 score is 984.0 and game length is 121
Game 152
Max tile after game 152 is 128.0 score is 1300.0 and game length is 143
Game 153
Max tile after game 153 is 128.0 score is 1380.0 and game length is 147
Game 154
Max tile after game 154 is 64.0 score is 684.0 and game length is 91
Game 155
Max tile after game 155 is 64.0 score is 1048.0 and game length is 128
Game 156
Max tile after game 156 is 128.0 score is 1296.0 and game length is 145
Game 157
Max tile after game 157 is 256.0 score is 2484.0 and game length is 213
Game 158
Max tile after game 158 is 128.0 score is 1076.0 and game length is 117
Game 159
Max tile after game 159 is 64.0 score is 768.0 and game length is 101
Game 160
Max tile after game 160 is 128.0 score is 1192.0 and game length is 133
Game 161
Max tile after game 161 is 128.0 score is 1708.0 and game length is 174
Game 162
Max tile after game 162 is 256.0 score 

Max tile after game 199 is 64.0 score is 472.0 and game length is 71
Game 200
Max tile after game 200 is 64.0 score is 564.0 and game length is 77
Game 201
Max tile after game 201 is 128.0 score is 1420.0 and game length is 147
Game 202
Max tile after game 202 is 64.0 score is 896.0 and game length is 112
Game 203
Max tile after game 203 is 128.0 score is 1508.0 and game length is 163
Game 204
Max tile after game 204 is 64.0 score is 736.0 and game length is 100
Game 205
Max tile after game 205 is 32.0 score is 380.0 and game length is 65
Game 206
Max tile after game 206 is 128.0 score is 1760.0 and game length is 166
Game 207
Max tile after game 207 is 128.0 score is 1472.0 and game length is 152
Game 208
Max tile after game 208 is 64.0 score is 856.0 and game length is 116
Game 209
Max tile after game 209 is 128.0 score is 1404.0 and game length is 146
Game 210
Max tile after game 210 is 128.0 score is 1176.0 and game length is 129
Game 211
Max tile after game 211 is 32.0 score is 32

...

Max tile after game 297 is 64.0 score is 776.0 and game length is 93
Game 298
Max tile after game 298 is 64.0 score is 752.0 and game length is 97
Game 299
Max tile after game 299 is 256.0 score is 2292.0 and game length is 204
Game 300
Max tile after game 300 is 128.0 score is 1144.0 and game length is 125
Game 301
Max tile after game 301 is 128.0 score is 1372.0 and game length is 145
Game 302
Max tile after game 302 is 64.0 score is 692.0 and game length is 93
Game 303
Max tile after game 303 is 128.0 score is 1024.0 and game length is 111
Game 304
Max tile after game 304 is 128.0 score is 1616.0 and game length is 167
Game 305
Max tile after game 305 is 64.0 score is 892.0 and game length is 111
Game 306
Max tile after game 306 is 64.0 score is 984.0 and game length is 121
Game 307
Max tile after game 307 is 128.0 score is 956.0 and game length is 102
Game 308
Max tile after game 308 is 64.0 score is 672.0 and game length is 90
Game 309
Max tile after game 309 is 32.0 score is 424.

Max tile after game 346 is 128.0 score is 1384.0 and game length is 141
Game 347
Max tile after game 347 is 128.0 score is 1116.0 and game length is 125
Game 348
Max tile after game 348 is 128.0 score is 1012.0 and game length is 109
Game 349
Max tile after game 349 is 128.0 score is 1608.0 and game length is 171
Game 350
Max tile after game 350 is 128.0 score is 1968.0 and game length is 184
Game 351
Max tile after game 351 is 128.0 score is 1740.0 and game length is 164
Game 352
Max tile after game 352 is 128.0 score is 1244.0 and game length is 134
Game 353
Max tile after game 353 is 128.0 score is 1164.0 and game length is 127
Game 354
Max tile after game 354 is 256.0 score is 2600.0 and game length is 218
Game 355
Max tile after game 355 is 64.0 score is 892.0 and game length is 115
Game 356
Max tile after game 356 is 128.0 score is 1120.0 and game length is 121
Game 357
Max tile after game 357 is 128.0 score is 1184.0 and game length is 122
Game 358
Max tile after game 358 is 128

Max tile after game 395 is 32.0 score is 420.0 and game length is 67
Game 396
Max tile after game 396 is 32.0 score is 308.0 and game length is 58
Game 397
Max tile after game 397 is 128.0 score is 1464.0 and game length is 151
Game 398
Max tile after game 398 is 256.0 score is 2280.0 and game length is 194
Game 399
Max tile after game 399 is 128.0 score is 1544.0 and game length is 161
Game 400
Max tile after game 400 is 128.0 score is 1700.0 and game length is 172
Game 401
Max tile after game 401 is 64.0 score is 924.0 and game length is 117
Game 402
Max tile after game 402 is 256.0 score is 2456.0 and game length is 213
Game 403
Max tile after game 403 is 128.0 score is 1940.0 and game length is 182
Game 404
Max tile after game 404 is 64.0 score is 688.0 and game length is 92
Game 405
Max tile after game 405 is 128.0 score is 1200.0 and game length is 132
Game 406
Max tile after game 406 is 128.0 score is 1420.0 and game length is 146
Game 407
Max tile after game 407 is 64.0 score i

Max tile after game 444 is 128.0 score is 1212.0 and game length is 134
Game 445
Max tile after game 445 is 128.0 score is 1312.0 and game length is 136
Game 446
Max tile after game 446 is 64.0 score is 620.0 and game length is 80
Game 447
Max tile after game 447 is 128.0 score is 1092.0 and game length is 117
Game 448
Max tile after game 448 is 128.0 score is 1184.0 and game length is 129
Game 449
Max tile after game 449 is 128.0 score is 1556.0 and game length is 163
Game 450
Max tile after game 450 is 128.0 score is 1304.0 and game length is 145
Game 451
Max tile after game 451 is 32.0 score is 540.0 and game length is 86
Game 452
Max tile after game 452 is 64.0 score is 1088.0 and game length is 130
Game 453
Max tile after game 453 is 32.0 score is 256.0 and game length is 51
Game 454
Max tile after game 454 is 64.0 score is 576.0 and game length is 80
Game 455
Max tile after game 455 is 128.0 score is 1644.0 and game length is 164
Game 456
Max tile after game 456 is 128.0 score is

Max tile after game 493 is 128.0 score is 1564.0 and game length is 156
Game 494
Max tile after game 494 is 64.0 score is 612.0 and game length is 88
Game 495
Max tile after game 495 is 32.0 score is 368.0 and game length is 64
Game 496
Max tile after game 496 is 128.0 score is 1620.0 and game length is 167
Game 497
Max tile after game 497 is 64.0 score is 1024.0 and game length is 126
Game 498
Max tile after game 498 is 64.0 score is 796.0 and game length is 109
Game 499
Max tile after game 499 is 32.0 score is 360.0 and game length is 60
Game 500
Max tile after game 500 is 256.0 score is 2116.0 and game length is 184


In [3]:
for game_count in range(500, 550):
    print('Game', game_count + 1)
    state = game.initial_state()
    moved = True
    score = 0
    game_len = 0
    while not game.is_game_over(state):
        action = None
        if moved:
            game_len += 1
            turn_count = min(turn_count + 1, turns_until_min_epsilon)
            epsilon = 1 - 0.95 * turn_count / turns_until_min_epsilon
            state = game.do_a_random_spawn(state)
            if random.random() < epsilon:
                action = random.randrange(game.number_of_actions)
            else:
                action = net.predict([dnn.prepare_input(state)])[1][0]
        else:
            action = random.randrange(game.number_of_actions)

        new_state, moved, reward = game.action(state, action, copy=True)
        score += reward
        if moved:
            exp_buffer.add([dnn.prepare_input(state), action, reward, dnn.prepare_input(new_state)])
            state = new_state
        if not exp_buffer.is_empty():
            batch = exp_buffer.get_batch(batch_size)
            old_states = [i[0] for i in batch]
            new_states = [i[3] for i in batch]
            rewards = [i[2] for i in batch]
            actions = [i[1] for i in batch]
            Q_values, initial_predictions = net.predict(old_states)
            target_Q_values = net.max_Q(new_states) * discount_factor + rewards
            for i in range(batch_size):
                Q_values[i][actions[i]] = target_Q_values[i]
            loss, _ = net.train(old_states, Q_values)
    print('Max tile after game', game_count + 1, 'is', state.max(), 'score is', score, 'and game length is', game_len)

Game 501
Max tile after game 501 is 128.0 score is 1560.0 and game length is 164
Game 502
Max tile after game 502 is 128.0 score is 1340.0 and game length is 139
Game 503
Max tile after game 503 is 128.0 score is 1512.0 and game length is 154
Game 504
Max tile after game 504 is 64.0 score is 812.0 and game length is 98
Game 505
Max tile after game 505 is 128.0 score is 1368.0 and game length is 146
Game 506
Max tile after game 506 is 32.0 score is 412.0 and game length is 68
Game 507
Max tile after game 507 is 64.0 score is 596.0 and game length is 79
Game 508
Max tile after game 508 is 64.0 score is 720.0 and game length is 100
Game 509
Max tile after game 509 is 128.0 score is 1280.0 and game length is 138
Game 510
Max tile after game 510 is 64.0 score is 592.0 and game length is 84
Game 511
Max tile after game 511 is 128.0 score is 1720.0 and game length is 177
Game 512
Max tile after game 512 is 32.0 score is 480.0 and game length is 73
Game 513
Max tile after game 513 is 256.0 sco

Max tile after game 550 is 128.0 score is 1444.0 and game length is 153


In [4]:
for game_count in range(550, 1000):
    print('Game', game_count + 1)
    state = game.initial_state()
    moved = True
    score = 0
    game_len = 0
    while not game.is_game_over(state):
        action = None
        if moved:
            game_len += 1
            turn_count = min(turn_count + 1, turns_until_min_epsilon)
            epsilon = 1 - 0.95 * turn_count / turns_until_min_epsilon
            state = game.do_a_random_spawn(state)
            if random.random() < epsilon:
                action = random.randrange(game.number_of_actions)
            else:
                action = net.predict([dnn.prepare_input(state)])[1][0]
        else:
            action = random.randrange(game.number_of_actions)

        new_state, moved, reward = game.action(state, action, copy=True)
        score += reward
        if moved:
            exp_buffer.add([dnn.prepare_input(state), action, reward, dnn.prepare_input(new_state)])
            state = new_state
        if not exp_buffer.is_empty():
            batch = exp_buffer.get_batch(batch_size)
            old_states = [i[0] for i in batch]
            new_states = [i[3] for i in batch]
            rewards = [i[2] for i in batch]
            actions = [i[1] for i in batch]
            Q_values, initial_predictions = net.predict(old_states)
            target_Q_values = net.max_Q(new_states) * discount_factor + rewards
            for i in range(batch_size):
                Q_values[i][actions[i]] = target_Q_values[i]
            loss, _ = net.train(old_states, Q_values)
    print('Max tile after game', game_count + 1, 'is', state.max(), 'score is', score, 'and game length is', game_len)

Game 551
Max tile after game 551 is 128.0 score is 1188.0 and game length is 124
Game 552
Max tile after game 552 is 64.0 score is 836.0 and game length is 103
Game 553
Max tile after game 553 is 128.0 score is 1244.0 and game length is 135
Game 554
Max tile after game 554 is 128.0 score is 1376.0 and game length is 147
Game 555
Max tile after game 555 is 32.0 score is 348.0 and game length is 64
Game 556
Max tile after game 556 is 128.0 score is 1084.0 and game length is 116
Game 557
Max tile after game 557 is 32.0 score is 456.0 and game length is 72
Game 558
Max tile after game 558 is 64.0 score is 604.0 and game length is 86
Game 559
Max tile after game 559 is 64.0 score is 944.0 and game length is 123
Game 560
Max tile after game 560 is 128.0 score is 1392.0 and game length is 151
Game 561
Max tile after game 561 is 128.0 score is 1308.0 and game length is 136
Game 562
Max tile after game 562 is 128.0 score is 948.0 and game length is 102
Game 563
Max tile after game 563 is 128.0 

Max tile after game 600 is 128.0 score is 1636.0 and game length is 168
Game 601
Max tile after game 601 is 128.0 score is 1316.0 and game length is 146
Game 602
Max tile after game 602 is 64.0 score is 616.0 and game length is 89
Game 603
Max tile after game 603 is 256.0 score is 2472.0 and game length is 213
Game 604
Max tile after game 604 is 32.0 score is 528.0 and game length is 84
Game 605
Max tile after game 605 is 128.0 score is 1720.0 and game length is 176
Game 606
Max tile after game 606 is 64.0 score is 608.0 and game length is 85
Game 607
Max tile after game 607 is 32.0 score is 312.0 and game length is 55
Game 608
Max tile after game 608 is 64.0 score is 652.0 and game length is 86
Game 609
Max tile after game 609 is 64.0 score is 928.0 and game length is 116
Game 610
Max tile after game 610 is 64.0 score is 616.0 and game length is 87
Game 611
Max tile after game 611 is 256.0 score is 2344.0 and game length is 197
Game 612
Max tile after game 612 is 128.0 score is 1456.0

Max tile after game 649 is 128.0 score is 1356.0 and game length is 142
Game 650
Max tile after game 650 is 32.0 score is 368.0 and game length is 67
Game 651
Max tile after game 651 is 128.0 score is 1324.0 and game length is 139
Game 652
Max tile after game 652 is 64.0 score is 724.0 and game length is 97
Game 653
Max tile after game 653 is 128.0 score is 1480.0 and game length is 147
Game 654
Max tile after game 654 is 128.0 score is 1784.0 and game length is 174
Game 655
Max tile after game 655 is 64.0 score is 844.0 and game length is 110
Game 656
Max tile after game 656 is 256.0 score is 2124.0 and game length is 179
Game 657
Max tile after game 657 is 128.0 score is 1084.0 and game length is 122
Game 658
Max tile after game 658 is 128.0 score is 1092.0 and game length is 118
Game 659
Max tile after game 659 is 64.0 score is 724.0 and game length is 97
Game 660
Max tile after game 660 is 64.0 score is 700.0 and game length is 95
Game 661
Max tile after game 661 is 64.0 score is 7

Max tile after game 698 is 128.0 score is 1660.0 and game length is 170
Game 699
Max tile after game 699 is 64.0 score is 580.0 and game length is 82
Game 700
Max tile after game 700 is 128.0 score is 1480.0 and game length is 156
Game 701
Max tile after game 701 is 32.0 score is 340.0 and game length is 59
Game 702
Max tile after game 702 is 128.0 score is 1104.0 and game length is 113
Game 703
Max tile after game 703 is 64.0 score is 696.0 and game length is 94
Game 704
Max tile after game 704 is 64.0 score is 1120.0 and game length is 134
Game 705
Max tile after game 705 is 128.0 score is 1380.0 and game length is 148
Game 706
Max tile after game 706 is 64.0 score is 564.0 and game length is 78
Game 707
Max tile after game 707 is 32.0 score is 224.0 and game length is 46
Game 708
Max tile after game 708 is 64.0 score is 1160.0 and game length is 137
Game 709
Max tile after game 709 is 128.0 score is 2220.0 and game length is 216
Game 710
Max tile after game 710 is 256.0 score is 232

Max tile after game 747 is 256.0 score is 2632.0 and game length is 225
Game 748
Max tile after game 748 is 128.0 score is 1680.0 and game length is 171
Game 749
Max tile after game 749 is 64.0 score is 536.0 and game length is 76
Game 750
Max tile after game 750 is 64.0 score is 804.0 and game length is 104
Game 751
Max tile after game 751 is 256.0 score is 2380.0 and game length is 204
Game 752
Max tile after game 752 is 128.0 score is 1544.0 and game length is 161
Game 753
Max tile after game 753 is 128.0 score is 1344.0 and game length is 141
Game 754
Max tile after game 754 is 128.0 score is 1872.0 and game length is 180
Game 755
Max tile after game 755 is 32.0 score is 268.0 and game length is 51
Game 756
Max tile after game 756 is 64.0 score is 980.0 and game length is 117
Game 757
Max tile after game 757 is 128.0 score is 1468.0 and game length is 158
Game 758
Max tile after game 758 is 64.0 score is 524.0 and game length is 73
Game 759
Max tile after game 759 is 32.0 score is 

Max tile after game 796 is 128.0 score is 1344.0 and game length is 140
Game 797
Max tile after game 797 is 128.0 score is 1380.0 and game length is 147
Game 798
Max tile after game 798 is 64.0 score is 652.0 and game length is 88
Game 799
Max tile after game 799 is 128.0 score is 1420.0 and game length is 151
Game 800
Max tile after game 800 is 64.0 score is 576.0 and game length is 82
Game 801
Max tile after game 801 is 64.0 score is 652.0 and game length is 88
Game 802
Max tile after game 802 is 256.0 score is 2636.0 and game length is 221
Game 803
Max tile after game 803 is 128.0 score is 1384.0 and game length is 149
Game 804
Max tile after game 804 is 64.0 score is 936.0 and game length is 114
Game 805
Max tile after game 805 is 128.0 score is 1432.0 and game length is 153
Game 806
Max tile after game 806 is 64.0 score is 784.0 and game length is 101
Game 807
Max tile after game 807 is 64.0 score is 824.0 and game length is 109
Game 808
Max tile after game 808 is 64.0 score is 57

Max tile after game 845 is 256.0 score is 2256.0 and game length is 185
Game 846
Max tile after game 846 is 128.0 score is 1172.0 and game length is 122
Game 847
Max tile after game 847 is 128.0 score is 1508.0 and game length is 154
Game 848
Max tile after game 848 is 64.0 score is 828.0 and game length is 110
Game 849
Max tile after game 849 is 128.0 score is 1508.0 and game length is 152
Game 850
Max tile after game 850 is 256.0 score is 2464.0 and game length is 215
Game 851
Max tile after game 851 is 64.0 score is 812.0 and game length is 102
Game 852
Max tile after game 852 is 64.0 score is 728.0 and game length is 98
Game 853
Max tile after game 853 is 64.0 score is 636.0 and game length is 91
Game 854
Max tile after game 854 is 64.0 score is 744.0 and game length is 101
Game 855
Max tile after game 855 is 64.0 score is 636.0 and game length is 84
Game 856
Max tile after game 856 is 128.0 score is 1344.0 and game length is 140
Game 857
Max tile after game 857 is 64.0 score is 83

Max tile after game 894 is 32.0 score is 356.0 and game length is 62
Game 895
Max tile after game 895 is 128.0 score is 1296.0 and game length is 140
Game 896
Max tile after game 896 is 128.0 score is 1400.0 and game length is 145
Game 897
Max tile after game 897 is 128.0 score is 1516.0 and game length is 149
Game 898
Max tile after game 898 is 64.0 score is 796.0 and game length is 99
Game 899
Max tile after game 899 is 64.0 score is 1260.0 and game length is 142
Game 900
Max tile after game 900 is 64.0 score is 652.0 and game length is 88
Game 901
Max tile after game 901 is 128.0 score is 1364.0 and game length is 145
Game 902
Max tile after game 902 is 128.0 score is 1556.0 and game length is 164
Game 903
Max tile after game 903 is 32.0 score is 496.0 and game length is 77
Game 904
Max tile after game 904 is 128.0 score is 1788.0 and game length is 170
Game 905
Max tile after game 905 is 128.0 score is 1184.0 and game length is 131
Game 906
Max tile after game 906 is 32.0 score is 

Max tile after game 943 is 32.0 score is 544.0 and game length is 84
Game 944
Max tile after game 944 is 64.0 score is 572.0 and game length is 80
Game 945
Max tile after game 945 is 32.0 score is 380.0 and game length is 65
Game 946
Max tile after game 946 is 32.0 score is 256.0 and game length is 51
Game 947
Max tile after game 947 is 64.0 score is 888.0 and game length is 113
Game 948
Max tile after game 948 is 128.0 score is 1304.0 and game length is 143
Game 949
Max tile after game 949 is 128.0 score is 1464.0 and game length is 156
Game 950
Max tile after game 950 is 32.0 score is 428.0 and game length is 71
Game 951
Max tile after game 951 is 32.0 score is 352.0 and game length is 65
Game 952
Max tile after game 952 is 64.0 score is 912.0 and game length is 113
Game 953
Max tile after game 953 is 64.0 score is 960.0 and game length is 119
Game 954
Max tile after game 954 is 64.0 score is 784.0 and game length is 103
Game 955
Max tile after game 955 is 128.0 score is 1428.0 and g

Max tile after game 992 is 128.0 score is 1028.0 and game length is 108
Game 993
Max tile after game 993 is 32.0 score is 332.0 and game length is 57
Game 994
Max tile after game 994 is 128.0 score is 1300.0 and game length is 133
Game 995
Max tile after game 995 is 64.0 score is 792.0 and game length is 101
Game 996
Max tile after game 996 is 64.0 score is 936.0 and game length is 117
Game 997
Max tile after game 997 is 64.0 score is 560.0 and game length is 80
Game 998
Max tile after game 998 is 64.0 score is 940.0 and game length is 115
Game 999
Max tile after game 999 is 128.0 score is 1276.0 and game length is 131
Game 1000
Max tile after game 1000 is 64.0 score is 796.0 and game length is 104


In [5]:
data = []
for i in range(1000):
  print('Game', i + 1)
  score = 0
  moved = True
  steps = 0
  state = game.initial_state()
  while not game.is_game_over(state):
    action = None
    if moved:
      steps += 1
      state = game.do_a_random_spawn(state)
      action = net.predict([dnn.prepare_input(state)])[1][0]
    else:
      action = random.randrange(game.number_of_actions)
    state, moved, reward = game.action(state, action)
    score += reward
  data.append([steps, state.max(), score])     
print(data)

Game 1
Game 2
Game 3
Game 4
Game 5
Game 6
Game 7
Game 8
Game 9
Game 10
Game 11
Game 12
Game 13
Game 14
Game 15
Game 16
Game 17
Game 18
Game 19
Game 20
Game 21
Game 22
Game 23
Game 24
Game 25
Game 26
Game 27
Game 28
Game 29
Game 30
Game 31
Game 32
Game 33
Game 34
Game 35
Game 36
Game 37
Game 38
Game 39
Game 40
Game 41
Game 42
Game 43
Game 44
Game 45
Game 46
Game 47
Game 48
Game 49
Game 50
Game 51
Game 52
Game 53
Game 54
Game 55
Game 56
Game 57
Game 58
Game 59
Game 60
Game 61
Game 62
Game 63
Game 64
Game 65
Game 66
Game 67
Game 68
Game 69
Game 70
Game 71
Game 72
Game 73
Game 74
Game 75
Game 76
Game 77
Game 78
Game 79
Game 80
Game 81
Game 82
Game 83
Game 84
Game 85
Game 86
Game 87
Game 88
Game 89
Game 90
Game 91
Game 92
Game 93
Game 94
Game 95
Game 96
Game 97
Game 98
Game 99
Game 100
Game 101
Game 102
Game 103
Game 104
Game 105
Game 106
Game 107
Game 108
Game 109
Game 110
Game 111
Game 112
Game 113
Game 114
Game 115
Game 116
Game 117
Game 118
Game 119
Game 120
Game 121
Game 122
Game 123
G

Game 158
Game 159
Game 160
Game 161
Game 162
Game 163
Game 164
Game 165
Game 166
Game 167
Game 168
Game 169
Game 170
Game 171
Game 172
Game 173
Game 174
Game 175
Game 176
Game 177
Game 178
Game 179
Game 180
Game 181
Game 182
Game 183
Game 184
Game 185
Game 186
Game 187
Game 188
Game 189
Game 190
Game 191
Game 192
Game 193
Game 194
Game 195
Game 196
Game 197
Game 198
Game 199
Game 200
Game 201
Game 202
Game 203
Game 204
Game 205
Game 206
Game 207
Game 208
Game 209
Game 210
Game 211
Game 212
Game 213
Game 214
Game 215
Game 216
Game 217
Game 218
Game 219
Game 220
Game 221
Game 222
Game 223
Game 224
Game 225
Game 226
Game 227
Game 228
Game 229
Game 230
Game 231
Game 232
Game 233
Game 234
Game 235
Game 236
Game 237
Game 238
Game 239
Game 240
Game 241
Game 242
Game 243
Game 244
Game 245
Game 246
Game 247
Game 248
Game 249
Game 250
Game 251
Game 252
Game 253
Game 254
Game 255
Game 256
Game 257
Game 258
Game 259
Game 260
Game 261
Game 262
Game 263
Game 264
Game 265
Game 266
Game 267
Game 268
G

Game 306
Game 307
Game 308
Game 309
Game 310
Game 311
Game 312
Game 313
Game 314
Game 315
Game 316
Game 317
Game 318
Game 319
Game 320
Game 321
Game 322
Game 323
Game 324
Game 325
Game 326
Game 327
Game 328
Game 329
Game 330
Game 331
Game 332
Game 333
Game 334
Game 335
Game 336
Game 337
Game 338
Game 339
Game 340
Game 341
Game 342
Game 343
Game 344
Game 345
Game 346
Game 347
Game 348
Game 349
Game 350
Game 351
Game 352
Game 353
Game 354
Game 355
Game 356
Game 357
Game 358
Game 359
Game 360
Game 361
Game 362
Game 363
Game 364
Game 365
Game 366
Game 367
Game 368
Game 369
Game 370
Game 371
Game 372
Game 373
Game 374
Game 375
Game 376
Game 377
Game 378
Game 379
Game 380
Game 381
Game 382
Game 383
Game 384
Game 385
Game 386
Game 387
Game 388
Game 389
Game 390
Game 391
Game 392
Game 393
Game 394
Game 395
Game 396
Game 397
Game 398
Game 399
Game 400
Game 401
Game 402
Game 403
Game 404
Game 405
Game 406
Game 407
Game 408
Game 409
Game 410
Game 411
Game 412
Game 413
Game 414
Game 415
Game 416
G

Game 454
Game 455
Game 456
Game 457
Game 458
Game 459
Game 460
Game 461
Game 462
Game 463
Game 464
Game 465
Game 466
Game 467
Game 468
Game 469
Game 470
Game 471
Game 472
Game 473
Game 474
Game 475
Game 476
Game 477
Game 478
Game 479
Game 480
Game 481
Game 482
Game 483
Game 484
Game 485
Game 486
Game 487
Game 488
Game 489
Game 490
Game 491
Game 492
Game 493
Game 494
Game 495
Game 496
Game 497
Game 498
Game 499
Game 500
Game 501
Game 502
Game 503
Game 504
Game 505
Game 506
Game 507
Game 508
Game 509
Game 510
Game 511
Game 512
Game 513
Game 514
Game 515
Game 516
Game 517
Game 518
Game 519
Game 520
Game 521
Game 522
Game 523
Game 524
Game 525
Game 526
Game 527
Game 528
Game 529
Game 530
Game 531
Game 532
Game 533
Game 534
Game 535
Game 536
Game 537
Game 538
Game 539
Game 540
Game 541
Game 542
Game 543
Game 544
Game 545
Game 546
Game 547
Game 548
Game 549
Game 550
Game 551
Game 552
Game 553
Game 554
Game 555
Game 556
Game 557
Game 558
Game 559
Game 560
Game 561
Game 562
Game 563
Game 564
G

Game 602
Game 603
Game 604
Game 605
Game 606
Game 607
Game 608
Game 609
Game 610
Game 611
Game 612
Game 613
Game 614
Game 615
Game 616
Game 617
Game 618
Game 619
Game 620
Game 621
Game 622
Game 623
Game 624
Game 625
Game 626
Game 627
Game 628
Game 629
Game 630
Game 631
Game 632
Game 633
Game 634
Game 635
Game 636
Game 637
Game 638
Game 639
Game 640
Game 641
Game 642
Game 643
Game 644
Game 645
Game 646
Game 647
Game 648
Game 649
Game 650
Game 651
Game 652
Game 653
Game 654
Game 655
Game 656
Game 657
Game 658
Game 659
Game 660
Game 661
Game 662
Game 663
Game 664
Game 665
Game 666
Game 667
Game 668
Game 669
Game 670
Game 671
Game 672
Game 673
Game 674
Game 675
Game 676
Game 677
Game 678
Game 679
Game 680
Game 681
Game 682
Game 683
Game 684
Game 685
Game 686
Game 687
Game 688
Game 689
Game 690
Game 691
Game 692
Game 693
Game 694
Game 695
Game 696
Game 697
Game 698
Game 699
Game 700
Game 701
Game 702
Game 703
Game 704
Game 705
Game 706
Game 707
Game 708
Game 709
Game 710
Game 711
Game 712
G

Game 750
Game 751
Game 752
Game 753
Game 754
Game 755
Game 756
Game 757
Game 758
Game 759
Game 760
Game 761
Game 762
Game 763
Game 764
Game 765
Game 766
Game 767
Game 768
Game 769
Game 770
Game 771
Game 772
Game 773
Game 774
Game 775
Game 776
Game 777
Game 778
Game 779
Game 780
Game 781
Game 782
Game 783
Game 784
Game 785
Game 786
Game 787
Game 788
Game 789
Game 790
Game 791
Game 792
Game 793
Game 794
Game 795
Game 796
Game 797
Game 798
Game 799
Game 800
Game 801
Game 802
Game 803
Game 804
Game 805
Game 806
Game 807
Game 808
Game 809
Game 810
Game 811
Game 812
Game 813
Game 814
Game 815
Game 816
Game 817
Game 818
Game 819
Game 820
Game 821
Game 822
Game 823
Game 824
Game 825
Game 826
Game 827
Game 828
Game 829
Game 830
Game 831
Game 832
Game 833
Game 834
Game 835
Game 836
Game 837
Game 838
Game 839
Game 840
Game 841
Game 842
Game 843
Game 844
Game 845
Game 846
Game 847
Game 848
Game 849
Game 850
Game 851
Game 852
Game 853
Game 854
Game 855
Game 856
Game 857
Game 858
Game 859
Game 860
G

Game 898
Game 899
Game 900
Game 901
Game 902
Game 903
Game 904
Game 905
Game 906
Game 907
Game 908
Game 909
Game 910
Game 911
Game 912
Game 913
Game 914
Game 915
Game 916
Game 917
Game 918
Game 919
Game 920
Game 921
Game 922
Game 923
Game 924
Game 925
Game 926
Game 927
Game 928
Game 929
Game 930
Game 931
Game 932
Game 933
Game 934
Game 935
Game 936
Game 937
Game 938
Game 939
Game 940
Game 941
Game 942
Game 943
Game 944
Game 945
Game 946
Game 947
Game 948
Game 949
Game 950
Game 951
Game 952
Game 953
Game 954
Game 955
Game 956
Game 957
Game 958
Game 959
Game 960
Game 961
Game 962
Game 963
Game 964
Game 965
Game 966
Game 967
Game 968
Game 969
Game 970
Game 971
Game 972
Game 973
Game 974
Game 975
Game 976
Game 977
Game 978
Game 979
Game 980
Game 981
Game 982
Game 983
Game 984
Game 985
Game 986
Game 987
Game 988
Game 989
Game 990
Game 991
Game 992
Game 993
Game 994
Game 995
Game 996
Game 997
Game 998
Game 999
Game 1000
[[113, 64.0, 860.0], [117, 128.0, 1100.0], [143, 128.0, 1388.0], [162, 1

In [6]:
losses = []
for game_count in range(1000, 1500):
    print('Game', game_count + 1)
    state = game.initial_state()
    moved = True
    score = 0
    game_len = 0
    while not game.is_game_over(state):
        action = None
        if moved:
            game_len += 1
            turn_count = min(turn_count + 1, turns_until_min_epsilon)
            epsilon = 1 - 0.95 * turn_count / turns_until_min_epsilon
            state = game.do_a_random_spawn(state)
            if random.random() < epsilon:
                action = random.randrange(game.number_of_actions)
            else:
                action = net.predict([dnn.prepare_input(state)])[1][0]
        else:
            action = random.randrange(game.number_of_actions)

        new_state, moved, reward = game.action(state, action, copy=True)
        score += reward
        if moved:
            exp_buffer.add([dnn.prepare_input(state), action, reward, dnn.prepare_input(new_state)])
            state = new_state
        if not exp_buffer.is_empty():
            batch = exp_buffer.get_batch(batch_size)
            old_states = [i[0] for i in batch]
            new_states = [i[3] for i in batch]
            rewards = [i[2] for i in batch]
            actions = [i[1] for i in batch]
            Q_values, initial_predictions = net.predict(old_states)
            target_Q_values = net.max_Q(new_states) * discount_factor + rewards
            for i in range(batch_size):
                Q_values[i][actions[i]] = target_Q_values[i]
            loss, _ = net.train(old_states, Q_values)
            losses.append(loss)
    print('Max tile after game', game_count + 1, 'is', state.max(), 'score is', score, 'and game length is', game_len)
print(losses)

Game 1001
Max tile after game 1001 is 64.0 score is 972.0 and game length is 119
Game 1002
Max tile after game 1002 is 128.0 score is 1424.0 and game length is 151
Game 1003
Max tile after game 1003 is 128.0 score is 1212.0 and game length is 131
Game 1004
Max tile after game 1004 is 128.0 score is 1776.0 and game length is 173
Game 1005
Max tile after game 1005 is 128.0 score is 1320.0 and game length is 138
Game 1006
Max tile after game 1006 is 64.0 score is 652.0 and game length is 87
Game 1007
Max tile after game 1007 is 64.0 score is 816.0 and game length is 108
Game 1008
Max tile after game 1008 is 64.0 score is 708.0 and game length is 100
Game 1009
Max tile after game 1009 is 64.0 score is 836.0 and game length is 114
Game 1010
Max tile after game 1010 is 256.0 score is 2940.0 and game length is 242
Game 1011
Max tile after game 1011 is 256.0 score is 2384.0 and game length is 206
Game 1012
Max tile after game 1012 is 128.0 score is 1444.0 and game length is 151
Game 1013
Max t

Max tile after game 1049 is 128.0 score is 1308.0 and game length is 141
Game 1050
Max tile after game 1050 is 64.0 score is 624.0 and game length is 88
Game 1051
Max tile after game 1051 is 64.0 score is 696.0 and game length is 94
Game 1052
Max tile after game 1052 is 128.0 score is 1412.0 and game length is 145
Game 1053
Max tile after game 1053 is 128.0 score is 1468.0 and game length is 155
Game 1054
Max tile after game 1054 is 128.0 score is 1024.0 and game length is 111
Game 1055
Max tile after game 1055 is 128.0 score is 1216.0 and game length is 129
Game 1056
Max tile after game 1056 is 128.0 score is 1088.0 and game length is 117
Game 1057
Max tile after game 1057 is 64.0 score is 576.0 and game length is 81
Game 1058
Max tile after game 1058 is 128.0 score is 1112.0 and game length is 116
Game 1059
Max tile after game 1059 is 128.0 score is 1444.0 and game length is 152
Game 1060
Max tile after game 1060 is 128.0 score is 1156.0 and game length is 128
Game 1061
Max tile afte

Max tile after game 1097 is 128.0 score is 940.0 and game length is 103
Game 1098
Max tile after game 1098 is 32.0 score is 564.0 and game length is 89
Game 1099
Max tile after game 1099 is 64.0 score is 708.0 and game length is 97
Game 1100
Max tile after game 1100 is 64.0 score is 612.0 and game length is 88
Game 1101
Max tile after game 1101 is 128.0 score is 1788.0 and game length is 171
Game 1102
Max tile after game 1102 is 64.0 score is 656.0 and game length is 90
Game 1103
Max tile after game 1103 is 128.0 score is 1032.0 and game length is 114
Game 1104
Max tile after game 1104 is 128.0 score is 1568.0 and game length is 166
Game 1105
Max tile after game 1105 is 64.0 score is 884.0 and game length is 109
Game 1106
Max tile after game 1106 is 32.0 score is 404.0 and game length is 67
Game 1107
Max tile after game 1107 is 32.0 score is 388.0 and game length is 66
Game 1108
Max tile after game 1108 is 64.0 score is 872.0 and game length is 112
Game 1109
Max tile after game 1109 is

Max tile after game 1145 is 128.0 score is 1460.0 and game length is 152
Game 1146
Max tile after game 1146 is 128.0 score is 1296.0 and game length is 138
Game 1147
Max tile after game 1147 is 128.0 score is 1412.0 and game length is 145
Game 1148
Max tile after game 1148 is 64.0 score is 820.0 and game length is 108
Game 1149
Max tile after game 1149 is 128.0 score is 1116.0 and game length is 122
Game 1150
Max tile after game 1150 is 64.0 score is 680.0 and game length is 91
Game 1151
Max tile after game 1151 is 64.0 score is 1116.0 and game length is 131
Game 1152
Max tile after game 1152 is 128.0 score is 1488.0 and game length is 159
Game 1153
Max tile after game 1153 is 128.0 score is 2044.0 and game length is 191
Game 1154
Max tile after game 1154 is 256.0 score is 2696.0 and game length is 221
Game 1155
Max tile after game 1155 is 64.0 score is 712.0 and game length is 93
Game 1156
Max tile after game 1156 is 32.0 score is 372.0 and game length is 63
Game 1157
Max tile after g

Max tile after game 1193 is 128.0 score is 1256.0 and game length is 130
Game 1194
Max tile after game 1194 is 128.0 score is 1264.0 and game length is 133
Game 1195
Max tile after game 1195 is 64.0 score is 804.0 and game length is 101
Game 1196
Max tile after game 1196 is 64.0 score is 540.0 and game length is 74
Game 1197
Max tile after game 1197 is 64.0 score is 628.0 and game length is 87
Game 1198
Max tile after game 1198 is 128.0 score is 1668.0 and game length is 168
Game 1199
Max tile after game 1199 is 128.0 score is 1484.0 and game length is 156
Game 1200
Max tile after game 1200 is 128.0 score is 1416.0 and game length is 153
Game 1201
Max tile after game 1201 is 128.0 score is 1204.0 and game length is 125
Game 1202
Max tile after game 1202 is 32.0 score is 200.0 and game length is 44
Game 1203
Max tile after game 1203 is 64.0 score is 744.0 and game length is 97
Game 1204
Max tile after game 1204 is 64.0 score is 468.0 and game length is 71
Game 1205
Max tile after game 1

...

Max tile after game 1338 is 128.0 score is 1656.0 and game length is 168
Game 1339
Max tile after game 1339 is 128.0 score is 1308.0 and game length is 134
Game 1340
Max tile after game 1340 is 256.0 score is 2368.0 and game length is 206
Game 1341
Max tile after game 1341 is 128.0 score is 1224.0 and game length is 132
Game 1342
Max tile after game 1342 is 32.0 score is 340.0 and game length is 62
Game 1343
Max tile after game 1343 is 64.0 score is 700.0 and game length is 95
Game 1344
Max tile after game 1344 is 128.0 score is 1600.0 and game length is 160
Game 1345
Max tile after game 1345 is 256.0 score is 2860.0 and game length is 253
Game 1346
Max tile after game 1346 is 32.0 score is 380.0 and game length is 65
Game 1347
Max tile after game 1347 is 64.0 score is 608.0 and game length is 87
Game 1348
Max tile after game 1348 is 128.0 score is 1348.0 and game length is 143
Game 1349
Max tile after game 1349 is 256.0 score is 2868.0 and game length is 237
Game 1350
Max tile after g

Max tile after game 1386 is 128.0 score is 1432.0 and game length is 150
Game 1387
Max tile after game 1387 is 256.0 score is 2340.0 and game length is 198
Game 1388
Max tile after game 1388 is 64.0 score is 792.0 and game length is 105
Game 1389
Max tile after game 1389 is 256.0 score is 2656.0 and game length is 226
Game 1390
Max tile after game 1390 is 256.0 score is 1892.0 and game length is 155
Game 1391
Max tile after game 1391 is 128.0 score is 1076.0 and game length is 114
Game 1392
Max tile after game 1392 is 64.0 score is 580.0 and game length is 82
Game 1393
Max tile after game 1393 is 128.0 score is 1116.0 and game length is 124
Game 1394
Max tile after game 1394 is 128.0 score is 1548.0 and game length is 158
Game 1395
Max tile after game 1395 is 64.0 score is 744.0 and game length is 102
Game 1396
Max tile after game 1396 is 64.0 score is 684.0 and game length is 92
Game 1397
Max tile after game 1397 is 128.0 score is 1452.0 and game length is 149
Game 1398
Max tile after

Max tile after game 1434 is 64.0 score is 808.0 and game length is 106
Game 1435
Max tile after game 1435 is 64.0 score is 772.0 and game length is 104
Game 1436
Max tile after game 1436 is 64.0 score is 708.0 and game length is 97
Game 1437
Max tile after game 1437 is 128.0 score is 1368.0 and game length is 140
Game 1438
Max tile after game 1438 is 64.0 score is 668.0 and game length is 93
Game 1439
Max tile after game 1439 is 128.0 score is 1060.0 and game length is 116
Game 1440
Max tile after game 1440 is 128.0 score is 1404.0 and game length is 144
Game 1441
Max tile after game 1441 is 64.0 score is 1084.0 and game length is 127
Game 1442
Max tile after game 1442 is 128.0 score is 1320.0 and game length is 148
Game 1443
Max tile after game 1443 is 64.0 score is 772.0 and game length is 101
Game 1444
Max tile after game 1444 is 64.0 score is 724.0 and game length is 100
Game 1445
Max tile after game 1445 is 64.0 score is 1028.0 and game length is 127
Game 1446
Max tile after game 

Max tile after game 1482 is 128.0 score is 1368.0 and game length is 144
Game 1483
Max tile after game 1483 is 128.0 score is 1112.0 and game length is 121
Game 1484
Max tile after game 1484 is 128.0 score is 1404.0 and game length is 150
Game 1485
Max tile after game 1485 is 64.0 score is 832.0 and game length is 111
Game 1486
Max tile after game 1486 is 64.0 score is 728.0 and game length is 98
Game 1487
Max tile after game 1487 is 64.0 score is 824.0 and game length is 100
Game 1488
Max tile after game 1488 is 64.0 score is 644.0 and game length is 93
Game 1489
Max tile after game 1489 is 128.0 score is 1256.0 and game length is 134
Game 1490
Max tile after game 1490 is 128.0 score is 1352.0 and game length is 145
Game 1491
Max tile after game 1491 is 64.0 score is 696.0 and game length is 94
Game 1492
Max tile after game 1492 is 256.0 score is 2280.0 and game length is 197
Game 1493
Max tile after game 1493 is 128.0 score is 1256.0 and game length is 134
Game 1494
Max tile after ga

Max tile after game 1500 is 128.0 score is 1044.0 and game length is 113
[239.01428, 734.2172, 266.7657, 532.1704, 318.8634, 404.49152, 641.07025, 488.1029, 251.26218, 292.48444, 217.78851, 233.5049, 350.94513, 479.8538, 186.10031, 5379.0137, 349.58405, 1356.4229, 172.36859, 387.58167, 413.06525, 110.56787, 340.97656, 342.5022, 405.81274, 3162.3284, 377.6294, 436.50684, 287.8745, 261.20572, 354.6247, 327.62, 252.91586, 776.6559, 378.3562, 557.45886, 284.83743, 378.88922, 182.52869, 8536.864, 5920.7676, 259.1549, 2301.7646, 1791.896, 64.25126, 453.86148, 449.4477, 213.0368, 199.69626, 286.84396, 272.27347, 551.1464, 405.0848, 338.69788, 183.57877, 7631.927, 381.977, 346.53586, 365.44415, 538.2849, 127.77471, 4547.4155, 104.76291, 264.25058, 383.23352, 251.81635, 234.6404, 1760.5624, 526.50867, 120.274475, 332.34436, 289.23987, 471.4992, 271.30353, 206.84595, 390.29706, 279.62567, 266.33624, 202.8599, 633.1703, 379.92197, 292.25226, 192.27473, 303.4345, 1415.1891, 362.21298, 292.51367, 4

In [7]:
for game_count in range(1500, 2000):
    print('Game', game_count + 1)
    state = game.initial_state()
    moved = True
    score = 0
    game_len = 0
    while not game.is_game_over(state):
        action = None
        if moved:
            game_len += 1
            turn_count = min(turn_count + 1, turns_until_min_epsilon)
            epsilon = 1 - 0.95 * turn_count / turns_until_min_epsilon
            state = game.do_a_random_spawn(state)
            if random.random() < epsilon:
                action = random.randrange(game.number_of_actions)
            else:
                action = net.predict([dnn.prepare_input(state)])[1][0]
        else:
            action = random.randrange(game.number_of_actions)

        new_state, moved, reward = game.action(state, action, copy=True)
        score += reward
        if moved:
            exp_buffer.add([dnn.prepare_input(state), action, reward, dnn.prepare_input(new_state)])
            state = new_state
        if not exp_buffer.is_empty():
            batch = exp_buffer.get_batch(batch_size)
            old_states = [i[0] for i in batch]
            new_states = [i[3] for i in batch]
            rewards = [i[2] for i in batch]
            actions = [i[1] for i in batch]
            Q_values, initial_predictions = net.predict(old_states)
            target_Q_values = net.max_Q(new_states) * discount_factor + rewards
            for i in range(batch_size):
                Q_values[i][actions[i]] = target_Q_values[i]
            loss, _ = net.train(old_states, Q_values)
            losses.append(loss)
    print('Max tile after game', game_count + 1, 'is', state.max(), 'score is', score, 'and game length is', game_len)
print(losses)

Game 1501
Max tile after game 1501 is 128.0 score is 1548.0 and game length is 156
Game 1502
Max tile after game 1502 is 64.0 score is 664.0 and game length is 90
Game 1503
Max tile after game 1503 is 64.0 score is 1136.0 and game length is 138
Game 1504
Max tile after game 1504 is 64.0 score is 676.0 and game length is 93
Game 1505
Max tile after game 1505 is 32.0 score is 428.0 and game length is 71
Game 1506
Max tile after game 1506 is 128.0 score is 1532.0 and game length is 163
Game 1507
Max tile after game 1507 is 64.0 score is 792.0 and game length is 106
Game 1508
Max tile after game 1508 is 256.0 score is 2380.0 and game length is 206
Game 1509
Max tile after game 1509 is 64.0 score is 824.0 and game length is 103
Game 1510
Max tile after game 1510 is 128.0 score is 1400.0 and game length is 146
Game 1511
Max tile after game 1511 is 64.0 score is 648.0 and game length is 90
Game 1512
Max tile after game 1512 is 64.0 score is 724.0 and game length is 99
Game 1513
Max tile after

Max tile after game 1549 is 64.0 score is 792.0 and game length is 106
Game 1550
Max tile after game 1550 is 128.0 score is 960.0 and game length is 102
Game 1551
Max tile after game 1551 is 128.0 score is 1576.0 and game length is 158
Game 1552
Max tile after game 1552 is 256.0 score is 2828.0 and game length is 237
Game 1553
Max tile after game 1553 is 128.0 score is 1648.0 and game length is 171
Game 1554
Max tile after game 1554 is 128.0 score is 1368.0 and game length is 145
Game 1555
Max tile after game 1555 is 128.0 score is 1392.0 and game length is 145
Game 1556
Max tile after game 1556 is 64.0 score is 796.0 and game length is 105
Game 1557
Max tile after game 1557 is 256.0 score is 2412.0 and game length is 206
Game 1558
Max tile after game 1558 is 128.0 score is 1416.0 and game length is 145
Game 1559
Max tile after game 1559 is 128.0 score is 1764.0 and game length is 170
Game 1560
Max tile after game 1560 is 64.0 score is 648.0 and game length is 90
Game 1561
Max tile aft

Max tile after game 1597 is 64.0 score is 696.0 and game length is 95
Game 1598
Max tile after game 1598 is 64.0 score is 848.0 and game length is 104
Game 1599
Max tile after game 1599 is 128.0 score is 1240.0 and game length is 135
Game 1600
Max tile after game 1600 is 128.0 score is 1320.0 and game length is 141
Game 1601
Max tile after game 1601 is 256.0 score is 2384.0 and game length is 201
Game 1602
Max tile after game 1602 is 64.0 score is 788.0 and game length is 104
Game 1603
Max tile after game 1603 is 64.0 score is 840.0 and game length is 112
Game 1604
Max tile after game 1604 is 64.0 score is 704.0 and game length is 95
Game 1605
Max tile after game 1605 is 64.0 score is 552.0 and game length is 76
Game 1606
Max tile after game 1606 is 64.0 score is 640.0 and game length is 91
Game 1607
Max tile after game 1607 is 128.0 score is 1152.0 and game length is 127
Game 1608
Max tile after game 1608 is 32.0 score is 532.0 and game length is 79
Game 1609
Max tile after game 1609 

Max tile after game 1645 is 64.0 score is 828.0 and game length is 107
Game 1646
Max tile after game 1646 is 128.0 score is 1100.0 and game length is 120
Game 1647
Max tile after game 1647 is 256.0 score is 2176.0 and game length is 191
Game 1648
Max tile after game 1648 is 128.0 score is 1356.0 and game length is 146
Game 1649
Max tile after game 1649 is 128.0 score is 1452.0 and game length is 154
Game 1650
Max tile after game 1650 is 128.0 score is 1344.0 and game length is 140
Game 1651
Max tile after game 1651 is 64.0 score is 632.0 and game length is 89
Game 1652
Max tile after game 1652 is 64.0 score is 1120.0 and game length is 127
Game 1653
Max tile after game 1653 is 32.0 score is 320.0 and game length is 60
Game 1654
Max tile after game 1654 is 64.0 score is 1236.0 and game length is 146
Game 1655
Max tile after game 1655 is 64.0 score is 696.0 and game length is 94
Game 1656
Max tile after game 1656 is 128.0 score is 1216.0 and game length is 131
Game 1657
Max tile after ga

Max tile after game 1693 is 128.0 score is 2076.0 and game length is 192
Game 1694
Max tile after game 1694 is 64.0 score is 828.0 and game length is 106
Game 1695
Max tile after game 1695 is 64.0 score is 768.0 and game length is 103
Game 1696
Max tile after game 1696 is 64.0 score is 692.0 and game length is 92
Game 1697
Max tile after game 1697 is 32.0 score is 308.0 and game length is 59
Game 1698
Max tile after game 1698 is 128.0 score is 1324.0 and game length is 139
Game 1699
Max tile after game 1699 is 64.0 score is 980.0 and game length is 117
Game 1700
Max tile after game 1700 is 64.0 score is 944.0 and game length is 113
Game 1701
Max tile after game 1701 is 256.0 score is 3028.0 and game length is 249
Game 1702
Max tile after game 1702 is 128.0 score is 1100.0 and game length is 119
Game 1703
Max tile after game 1703 is 128.0 score is 1176.0 and game length is 129
Game 1704
Max tile after game 1704 is 64.0 score is 708.0 and game length is 96
Game 1705
Max tile after game 1

...

Max tile after game 1837 is 128.0 score is 1380.0 and game length is 148
Game 1838
Max tile after game 1838 is 32.0 score is 404.0 and game length is 69
Game 1839
Max tile after game 1839 is 128.0 score is 1236.0 and game length is 133
Game 1840
Max tile after game 1840 is 128.0 score is 1016.0 and game length is 110
Game 1841
Max tile after game 1841 is 128.0 score is 1336.0 and game length is 138
Game 1842
Max tile after game 1842 is 512.0 score is 4680.0 and game length is 321
Game 1843
Max tile after game 1843 is 64.0 score is 808.0 and game length is 107
Game 1844
Max tile after game 1844 is 64.0 score is 740.0 and game length is 101
Game 1845
Max tile after game 1845 is 256.0 score is 1904.0 and game length is 157
Game 1846
Max tile after game 1846 is 64.0 score is 716.0 and game length is 95
Game 1847
Max tile after game 1847 is 64.0 score is 516.0 and game length is 77
Game 1848
Max tile after game 1848 is 128.0 score is 1736.0 and game length is 163
Game 1849
Max tile after ga

Max tile after game 1886 is 64.0 score is 728.0 and game length is 99
Game 1887
Max tile after game 1887 is 128.0 score is 1076.0 and game length is 120
Game 1888
Max tile after game 1888 is 64.0 score is 884.0 and game length is 103
Game 1889
Max tile after game 1889 is 128.0 score is 1404.0 and game length is 143
Game 1890
Max tile after game 1890 is 128.0 score is 1320.0 and game length is 144
Game 1891
Max tile after game 1891 is 64.0 score is 844.0 and game length is 110
Game 1892
Max tile after game 1892 is 128.0 score is 1304.0 and game length is 135
Game 1893
Max tile after game 1893 is 128.0 score is 1056.0 and game length is 116
Game 1894
Max tile after game 1894 is 64.0 score is 824.0 and game length is 109
Game 1895
Max tile after game 1895 is 128.0 score is 1408.0 and game length is 148
Game 1896
Max tile after game 1896 is 128.0 score is 1044.0 and game length is 117
Game 1897
Max tile after game 1897 is 128.0 score is 1124.0 and game length is 122
Game 1898
Max tile afte

Max tile after game 1934 is 64.0 score is 664.0 and game length is 89
Game 1935
Max tile after game 1935 is 128.0 score is 1592.0 and game length is 158
Game 1936
Max tile after game 1936 is 128.0 score is 1072.0 and game length is 117
Game 1937
Max tile after game 1937 is 128.0 score is 1240.0 and game length is 128
Game 1938
Max tile after game 1938 is 32.0 score is 400.0 and game length is 69
Game 1939
Max tile after game 1939 is 256.0 score is 2652.0 and game length is 223
Game 1940
Max tile after game 1940 is 128.0 score is 1624.0 and game length is 162
Game 1941
Max tile after game 1941 is 128.0 score is 1372.0 and game length is 146
Game 1942
Max tile after game 1942 is 256.0 score is 2428.0 and game length is 206
Game 1943
Max tile after game 1943 is 128.0 score is 2220.0 and game length is 209
Game 1944
Max tile after game 1944 is 128.0 score is 1340.0 and game length is 139
Game 1945
Max tile after game 1945 is 128.0 score is 1320.0 and game length is 138
Game 1946
Max tile a

Max tile after game 1982 is 64.0 score is 532.0 and game length is 77
Game 1983
Max tile after game 1983 is 64.0 score is 716.0 and game length is 97
Game 1984
Max tile after game 1984 is 128.0 score is 1308.0 and game length is 145
Game 1985
Max tile after game 1985 is 32.0 score is 244.0 and game length is 51
Game 1986
Max tile after game 1986 is 128.0 score is 1676.0 and game length is 172
Game 1987
Max tile after game 1987 is 64.0 score is 684.0 and game length is 94
Game 1988
Max tile after game 1988 is 128.0 score is 1756.0 and game length is 180
Game 1989
Max tile after game 1989 is 32.0 score is 260.0 and game length is 51
Game 1990
Max tile after game 1990 is 128.0 score is 996.0 and game length is 106
Game 1991
Max tile after game 1991 is 64.0 score is 748.0 and game length is 99
Game 1992
Max tile after game 1992 is 32.0 score is 288.0 and game length is 56
Game 1993
Max tile after game 1993 is 128.0 score is 1292.0 and game length is 132
Game 1994
Max tile after game 1994 i

Max tile after game 2000 is 64.0 score is 900.0 and game length is 116
[239.01428, 734.2172, 266.7657, 532.1704, 318.8634, 404.49152, 641.07025, 488.1029, 251.26218, 292.48444, 217.78851, 233.5049, 350.94513, 479.8538, 186.10031, 5379.0137, 349.58405, 1356.4229, 172.36859, 387.58167, 413.06525, 110.56787, 340.97656, 342.5022, 405.81274, 3162.3284, 377.6294, 436.50684, 287.8745, 261.20572, 354.6247, 327.62, 252.91586, 776.6559, 378.3562, 557.45886, 284.83743, 378.88922, 182.52869, 8536.864, 5920.7676, 259.1549, 2301.7646, 1791.896, 64.25126, 453.86148, 449.4477, 213.0368, 199.69626, 286.84396, 272.27347, 551.1464, 405.0848, 338.69788, 183.57877, 7631.927, 381.977, 346.53586, 365.44415, 538.2849, 127.77471, 4547.4155, 104.76291, 264.25058, 383.23352, 251.81635, 234.6404, 1760.5624, 526.50867, 120.274475, 332.34436, 289.23987, 471.4992, 271.30353, 206.84595, 390.29706, 279.62567, 266.33624, 202.8599, 633.1703, 379.92197, 292.25226, 192.27473, 303.4345, 1415.1891, 362.21298, 292.51367, 415

In [8]:
data = []
for i in range(1000):
  print('Game', i + 1)
  score = 0
  moved = True
  steps = 0
  state = game.initial_state()
  while not game.is_game_over(state):
    action = None
    if moved:
      steps += 1
      state = game.do_a_random_spawn(state)
      action = net.predict([dnn.prepare_input(state)])[1][0]
    else:
      action = random.randrange(game.number_of_actions)
    state, moved, reward = game.action(state, action)
    score += reward
  data.append([steps, state.max(), score])     
print(data)

Game 1
Game 2
Game 3
Game 4
Game 5
Game 6
Game 7
Game 8
Game 9
Game 10
Game 11
Game 12
Game 13
Game 14
Game 15
Game 16
Game 17
Game 18
Game 19
Game 20
Game 21
Game 22
Game 23
Game 24
Game 25
Game 26
Game 27
Game 28
Game 29
Game 30
Game 31
Game 32
Game 33
Game 34
Game 35
Game 36
Game 37
Game 38
Game 39
Game 40
Game 41
Game 42
Game 43
Game 44
Game 45
Game 46
Game 47
Game 48
Game 49
Game 50
Game 51
Game 52
Game 53
Game 54
Game 55
Game 56
Game 57
Game 58
Game 59
Game 60
Game 61
Game 62
Game 63
Game 64
Game 65
Game 66
Game 67
Game 68
Game 69
Game 70
Game 71
Game 72
Game 73
Game 74
Game 75
Game 76
Game 77
Game 78
Game 79
Game 80
Game 81
Game 82
Game 83
Game 84
Game 85
Game 86
Game 87
Game 88
Game 89
Game 90
Game 91
Game 92
Game 93
Game 94
Game 95
Game 96
Game 97
Game 98
Game 99
Game 100
Game 101
Game 102
Game 103
Game 104
Game 105
Game 106
Game 107
Game 108
Game 109
Game 110
Game 111
Game 112
Game 113
Game 114
Game 115
Game 116
Game 117
Game 118
Game 119
Game 120
Game 121
Game 122
Game 123
G

Game 158
Game 159
Game 160
Game 161
Game 162
Game 163
Game 164
Game 165
Game 166
Game 167
Game 168
Game 169
Game 170
Game 171
Game 172
Game 173
Game 174
Game 175
Game 176
Game 177
Game 178
Game 179
Game 180
Game 181
Game 182
Game 183
Game 184
Game 185
Game 186
Game 187
Game 188
Game 189
Game 190
Game 191
Game 192
Game 193
Game 194
Game 195
Game 196
Game 197
Game 198
Game 199
Game 200
Game 201
Game 202
Game 203
Game 204
Game 205
Game 206
Game 207
Game 208
Game 209
Game 210
Game 211
Game 212
Game 213
Game 214
Game 215
Game 216
Game 217
Game 218
Game 219
Game 220
Game 221
Game 222
Game 223
Game 224
Game 225
Game 226
Game 227
Game 228
Game 229
Game 230
Game 231
Game 232
Game 233
Game 234
Game 235
Game 236
Game 237
Game 238
Game 239
Game 240
Game 241
Game 242
Game 243
Game 244
Game 245
Game 246
Game 247
Game 248
Game 249
Game 250
Game 251
Game 252
Game 253
Game 254
Game 255
Game 256
Game 257
Game 258
Game 259
Game 260
Game 261
Game 262
Game 263
Game 264
Game 265
Game 266
Game 267
Game 268
G

Game 306
Game 307
Game 308
Game 309
Game 310
Game 311
Game 312
Game 313
Game 314
Game 315
Game 316
Game 317
Game 318
Game 319
Game 320
Game 321
Game 322
Game 323
Game 324
Game 325
Game 326
Game 327
Game 328
Game 329
Game 330
Game 331
Game 332
Game 333
Game 334
Game 335
Game 336
Game 337
Game 338
Game 339
Game 340
Game 341
Game 342
Game 343
Game 344
Game 345
Game 346
Game 347
Game 348
Game 349
Game 350
Game 351
Game 352
Game 353
Game 354
Game 355
Game 356
Game 357
Game 358
Game 359
Game 360
Game 361
Game 362
Game 363
Game 364
Game 365
Game 366
Game 367
Game 368
Game 369
Game 370
Game 371
Game 372
Game 373
Game 374
Game 375
Game 376
Game 377
Game 378
Game 379
Game 380
Game 381
Game 382
Game 383
Game 384
Game 385
Game 386
Game 387
Game 388
Game 389
Game 390
Game 391
Game 392
Game 393
Game 394
Game 395
Game 396
Game 397
Game 398
Game 399
Game 400
Game 401
Game 402
Game 403
Game 404
Game 405
Game 406
Game 407
Game 408
Game 409
Game 410
Game 411
Game 412
Game 413
Game 414
Game 415
Game 416
G

Game 454
Game 455
Game 456
Game 457
Game 458
Game 459
Game 460
Game 461
Game 462
Game 463
Game 464
Game 465
Game 466
Game 467
Game 468
Game 469
Game 470
Game 471
Game 472
Game 473
Game 474
Game 475
Game 476
Game 477
Game 478
Game 479
Game 480
Game 481
Game 482
Game 483
Game 484
Game 485
Game 486
Game 487
Game 488
Game 489
Game 490
Game 491
Game 492
Game 493
Game 494
Game 495
Game 496
Game 497
Game 498
Game 499
Game 500
Game 501
Game 502
Game 503
Game 504
Game 505
Game 506
Game 507
Game 508
Game 509
Game 510
Game 511
Game 512
Game 513
Game 514
Game 515
Game 516
Game 517
Game 518
Game 519
Game 520
Game 521
Game 522
Game 523
Game 524
Game 525
Game 526
Game 527
Game 528
Game 529
Game 530
Game 531
Game 532
Game 533
Game 534
Game 535
Game 536
Game 537
Game 538
Game 539
Game 540
Game 541
Game 542
Game 543
Game 544
Game 545
Game 546
Game 547
Game 548
Game 549
Game 550
Game 551
Game 552
Game 553
Game 554
Game 555
Game 556
Game 557
Game 558
Game 559
Game 560
Game 561
Game 562
Game 563
Game 564
G

Game 602
Game 603
Game 604
Game 605
Game 606
Game 607
Game 608
Game 609
Game 610
Game 611
Game 612
Game 613
Game 614
Game 615
Game 616
Game 617
Game 618
Game 619
Game 620
Game 621
Game 622
Game 623
Game 624
Game 625
Game 626
Game 627
Game 628
Game 629
Game 630
Game 631
Game 632
Game 633
Game 634
Game 635
Game 636
Game 637
Game 638
Game 639
Game 640
Game 641
Game 642
Game 643
Game 644
Game 645
Game 646
Game 647
Game 648
Game 649
Game 650
Game 651
Game 652
Game 653
Game 654
Game 655
Game 656
Game 657
Game 658
Game 659
Game 660
Game 661
Game 662
Game 663
Game 664
Game 665
Game 666
Game 667
Game 668
Game 669
Game 670
Game 671
Game 672
Game 673
Game 674
Game 675
Game 676
Game 677
Game 678
Game 679
Game 680
Game 681
Game 682
Game 683
Game 684
Game 685
Game 686
Game 687
Game 688
Game 689
Game 690
Game 691
Game 692
Game 693
Game 694
Game 695
Game 696
Game 697
Game 698
Game 699
Game 700
Game 701
Game 702
Game 703
Game 704
Game 705
Game 706
Game 707
Game 708
Game 709
Game 710
Game 711
Game 712
G

Game 750
Game 751
Game 752
Game 753
Game 754
Game 755
Game 756
Game 757
Game 758
Game 759
Game 760
Game 761
Game 762
Game 763
Game 764
Game 765
Game 766
Game 767
Game 768
Game 769
Game 770
Game 771
Game 772
Game 773
Game 774
Game 775
Game 776
Game 777
Game 778
Game 779
Game 780
Game 781
Game 782
Game 783
Game 784
Game 785
Game 786
Game 787
Game 788
Game 789
Game 790
Game 791
Game 792
Game 793
Game 794
Game 795
Game 796
Game 797
Game 798
Game 799
Game 800
Game 801
Game 802
Game 803
Game 804
Game 805
Game 806
Game 807
Game 808
Game 809
Game 810
Game 811
Game 812
Game 813
Game 814
Game 815
Game 816
Game 817
Game 818
Game 819
Game 820
Game 821
Game 822
Game 823
Game 824
Game 825
Game 826
Game 827
Game 828
Game 829
Game 830
Game 831
Game 832
Game 833
Game 834
Game 835
Game 836
Game 837
Game 838
Game 839
Game 840
Game 841
Game 842
Game 843
Game 844
Game 845
Game 846
Game 847
Game 848
Game 849
Game 850
Game 851
Game 852
Game 853
Game 854
Game 855
Game 856
Game 857
Game 858
Game 859
Game 860
G

Game 898
Game 899
Game 900
Game 901
Game 902
Game 903
Game 904
Game 905
Game 906
Game 907
Game 908
Game 909
Game 910
Game 911
Game 912
Game 913
Game 914
Game 915
Game 916
Game 917
Game 918
Game 919
Game 920
Game 921
Game 922
Game 923
Game 924
Game 925
Game 926
Game 927
Game 928
Game 929
Game 930
Game 931
Game 932
Game 933
Game 934
Game 935
Game 936
Game 937
Game 938
Game 939
Game 940
Game 941
Game 942
Game 943
Game 944
Game 945
Game 946
Game 947
Game 948
Game 949
Game 950
Game 951
Game 952
Game 953
Game 954
Game 955
Game 956
Game 957
Game 958
Game 959
Game 960
Game 961
Game 962
Game 963
Game 964
Game 965
Game 966
Game 967
Game 968
Game 969
Game 970
Game 971
Game 972
Game 973
Game 974
Game 975
Game 976
Game 977
Game 978
Game 979
Game 980
Game 981
Game 982
Game 983
Game 984
Game 985
Game 986
Game 987
Game 988
Game 989
Game 990
Game 991
Game 992
Game 993
Game 994
Game 995
Game 996
Game 997
Game 998
Game 999
Game 1000
[[133, 128.0, 1264.0], [276, 256.0, 3332.0], [120, 128.0, 1136.0], [65, 